<a href="https://colab.research.google.com/github/hammad93/hurricane-tts/blob/main/hurricane_tts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation
Please make sure to run all tests.

In [1]:
!git clone https://github.com/hammad93/hurricane-tts.git

Cloning into 'hurricane-tts'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 66 (delta 32), reused 43 (delta 16), pack-reused 0
Receiving objects: 100% (66/66), 43.62 KiB | 1.74 MiB/s, done.
Resolving deltas: 100% (32/32), done.


In [2]:
!pip install -r hurricane-tts/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.2/220.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [1]:
import os
if not os.getenv("AZURE_OPENAI_API_KEY") :
  from google.colab import userdata
  os.environ["AZURE_OPENAI_API_KEY"] = userdata.get('AZURE_OPENAI_API_KEY')

In [2]:
%cd /content/hurricane-tts
!python test.py

/content/hurricane-tts
Here are the constructed messages: [{'role': 'system', 'content': 'You are an AI assistant that helps people find information.'}, {'role': 'user', 'content': 'test'}]
..
----------------------------------------------------------------------
Ran 2 tests in 5.245s

OK


In [3]:
import prompts
import utils
# we generate prompts by ingesting live hurricane data and supported languages
storm_data = utils.transform_storm_data()
prompt_data = prompts.generate_prompts()
supported_langs = prompts.unique_lang_list()

In [4]:
# example
print(prompt_data['storms'][0])

These are the input storm records and forecasts.
Each record has a lat and lon according to their geographic coordinates and wind speed in knots.
Please respond with the 3 most relevant languages exactly as they appear from the supported language list other than English each delimited by a comma.
[{'type': 'history', 'lat': 4.8, 'lon': 152.0, 'time': '2023-11-08 18:00:00', 'wind_speed': 15}, {'type': 'history', 'lat': 5.1, 'lon': 151.4, 'time': '2023-11-09 00:00:00', 'wind_speed': 15}, {'type': 'history', 'lat': 5.3, 'lon': 150.8, 'time': '2023-11-09 06:00:00', 'wind_speed': 15}, {'type': 'forecast', 'lat': 5.5, 'lon': 151.0, 'time': '2023-11-09T18:00:00', 'wind_speed': 20}, {'type': 'forecast', 'lat': 5.9, 'lon': 151.5, 'time': '2023-11-10T06:00:00', 'wind_speed': 25}, {'type': 'forecast', 'lat': 6.4, 'lon': 152.0, 'time': '2023-11-10T18:00:00', 'wind_speed': 30}, {'type': 'forecast', 'lat': 6.7, 'lon': 152.5, 'time': '2023-11-11T06:00:00', 'wind_speed': 35}, {'type': 'forecast', 'lat

# Language Geography inference from tropical storm geographical coorindates
We're trying to answer the question, "Which languages do we produce speech for to report on this tropical storm?" This pipeline will produce an English output but also utilize massively multilingual capabilities such that emergency notices are also in local languages

In [5]:
retries = 5 # sometimes it fails, so we retry it
storm_langs = {} # keys are storm id's and values are the languages
storm_chats = {} # stores chat histories for storms
for index, storm_prompt in enumerate(prompt_data['storms']) :
  storm_id = list(storm_data.keys())[index]
  # Get the languages to generate the report for this storm
  while retries > 0 :
    response = prompts.chat(system=prompt_data['system'], message=storm_prompt)
    print(response)
    result = utils.llm_response_transform(
        resp=response, supported_langs=supported_langs)
    print(result)
    if result :
      break
    else :
      retries = retries - 1
      print(f"Failed. Retries left: {retries}")
  if retries < 1 :
    raise Exception("Couldn't produce a correct output from LLM.")

  # store results
  storm_langs[storm_id] = {'names': ['English'] + result}
  storm_chats[storm_id] = {
      'history' : [{'role': 'system', 'content': prompt_data['system']},
                   {'role': 'user', 'content': storm_prompt},
                   {'role': 'assistant', 'content': response}]
  }

Here are the constructed messages: [{'role': 'system', 'content': "You are an expert in languages according to their geographical location.\nI have a system for emergency notification of tropical storms that utilizes official data sources and creates speech audio from text with a massively multilingual model.\nThis is the list of supported languages,\n{'Ditammari', 'Tado', 'Tohono Oâ\\x80\\x99odham', 'Epena', 'Tyap', 'Jola-Fonyi', 'Bonggi', 'Kwaio', 'Mixtec, Jamiltepec', 'Ngbaka', 'Nahuatl, Northern Oaxaca', 'Greek, Ancient', 'Tatar', 'Taabwa', 'Ron', 'Amharic', 'Gaddi', 'Kalinga, Lubuagan', 'Maya, MopÃ¡n', 'Maldivian', 'Mbore', 'Ayta, Mag-antsi', 'Yoruba', 'Luguru', 'Matsigenka', 'Haya', 'Bana', 'Maâ\\x80\\x99di', 'Ilocano', 'Marba', 'Javanese, Suriname', 'Sissala', 'Bisu', 'Karelian', 'Limba, West-Central', 'Balantak', 'Vietnamese', 'Huave, San Mateo del Mar', 'Mbuko', 'Loma', 'Buglere', 'Malay, Kupang', 'Selee', 'Aringa', 'Rabha', 'Tatuyo', 'Aghul', 'Dedua', 'Mende', 'Wanano', 'Zapo

# Tropical Storm Report
This code will create the report in the language specified based on the tropical storm.

In [7]:
with open('prompts/report-prompt.txt', 'r') as file:
  report_prompt = file.read()
for storm in storm_langs:
  reports = []
  for lang in storm_langs[storm]['names']:
    print(storm_chats[storm]['history'])
    # Construct the prompt
    message = report_prompt.format(lang=lang)
    print(message)
    # Send to LLM
    response = prompts.chat(
        message = message, history = storm_chats[storm]['history'])
    print(response)
    # store data
    reports.append(response)
  storm_langs[storm]['reports'] = reports

[{'role': 'system', 'content': "You are an expert in languages according to their geographical location.\nI have a system for emergency notification of tropical storms that utilizes official data sources and creates speech audio from text with a massively multilingual model.\nThis is the list of supported languages,\n{'Ditammari', 'Tado', 'Tohono Oâ\\x80\\x99odham', 'Epena', 'Tyap', 'Jola-Fonyi', 'Bonggi', 'Kwaio', 'Mixtec, Jamiltepec', 'Ngbaka', 'Nahuatl, Northern Oaxaca', 'Greek, Ancient', 'Tatar', 'Taabwa', 'Ron', 'Amharic', 'Gaddi', 'Kalinga, Lubuagan', 'Maya, MopÃ¡n', 'Maldivian', 'Mbore', 'Ayta, Mag-antsi', 'Yoruba', 'Luguru', 'Matsigenka', 'Haya', 'Bana', 'Maâ\\x80\\x99di', 'Ilocano', 'Marba', 'Javanese, Suriname', 'Sissala', 'Bisu', 'Karelian', 'Limba, West-Central', 'Balantak', 'Vietnamese', 'Huave, San Mateo del Mar', 'Mbuko', 'Loma', 'Buglere', 'Malay, Kupang', 'Selee', 'Aringa', 'Rabha', 'Tatuyo', 'Aghul', 'Dedua', 'Mende', 'Wanano', 'Zapotec, Texmelucan', 'Uzbek', 'Nahuatl

In [8]:
storm_langs

{'WP952023': {'names': ['English', 'Malay', 'Tagalog', 'Indonesian'],
  'reports': ['Attention, residents of [location]. A tropical storm is approaching the area. The storm is predicted to hit [location] at [time] with winds reaching up to [wind speed] knots. Please take necessary precautions and secure your homes and belongings. Follow local emergency protocols and stay tuned for further updates from local authorities. Stay safe and stay informed.',
   'Ini adalah laporan cuaca untuk kawasan ini. Kita sedang memantau gelombang tropika dengan kemasukan koordinat geografi sekitar latitud 4.8 dan longitud 152.0. Angin kencang diperhatikan dan akan terus dipantau. Ramalan menunjukkan badai akan terus bertambah kuat dengan angin yang dijangka akan mencapai 40 knot pada 12 November 2023. Mohon pastikan anda mengambil langkah-langkah keselamatan dan terus mengikuti perkembangan cuaca dari sumber rasmi.',
   'Babala ng bagyo. Sa kasalukuyan may aktibong bagyo sa may Pacific Ocean. May dalang 

The following are from the TTS implementation
https://github.com/facebookresearch/fairseq/blob/main/examples/mms/tts/tutorial/MMS_TTS_Inference_Colab.ipynb


In [ ]:
%cd ~/
!git clone https://github.com/jaywalnut310/vits.git
%cd vits/

!pip install Cython==0.29.21
!pip install librosa==0.8.0
!pip install phonemizer==2.2.1
!pip install scipy
!pip install numpy
!pip install torch
!pip install torchvision
!pip install matplotlib
!pip install Unidecode==1.1.1

%cd monotonic_align/
%mkdir monotonic_align
!python3 setup.py build_ext --inplace
%cd ../
%pwd

In [9]:
import sys
del sys.modules["utils"]

In [10]:
%cd /content/hurricane-tts/
import fair_mms_tts
%cd /root/vits
import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn

/content/hurricane-tts
/root/vits


In [19]:
lang_code = 'ind'
ckpt_dir = fair_mms_tts.download(lang_code)

Download model for language: ind
Model checkpoints in ./ind: ['vocab.txt', 'config.json', 'G_100000.pth']


In [12]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

cuda


In [22]:
print(f"Run inference with {device}")
vocab_file = f"{ckpt_dir}/vocab.txt"
config_file = f"{ckpt_dir}/config.json"
assert os.path.isfile(config_file), f"{config_file} doesn't exist"
hps = utils.get_hparams_from_file(config_file)

text_mapper = fair_mms_tts.TextMapper(vocab_file)
net_g = SynthesizerTrn(
    len(text_mapper.symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model)
net_g.to(device)
_ = net_g.eval()

g_pth = f"{ckpt_dir}/G_100000.pth"
print(f"load {g_pth}")

_ = utils.load_checkpoint(g_pth, net_g, None)

Run inference with cuda


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


load ./ind/G_100000.pth


In [23]:
txt = '''
Berikut adalah laporan cuaca terbaru. Saat ini, ada badai tropis terbentuk di Samudra Pasifik. Badai ini diperkirakan akan bergerak ke arah barat laut dan kemungkinan besar akan mempengaruhi Indonesia dalam beberapa hari ke depan. Badai ini diperkirakan akan menghasilkan angin kencang dengan kecepatan mencapai 35 knot atau 65 kilometer per jam. Di lokasi yang terkena dampak badai, kemungkinan besar akan terjadi hujan lebat, gelombang tinggi, dan banjir bandang. Kami menyarankan agar Anda mengambil langkah-langkah untuk mempersiapkan diri dan mengikuti informasi terbaru dari pihak berwenang. Terima kasih.
'''
print(txt)


Berikut adalah laporan cuaca terbaru. Saat ini, ada badai tropis terbentuk di Samudra Pasifik. Badai ini diperkirakan akan bergerak ke arah barat laut dan kemungkinan besar akan mempengaruhi Indonesia dalam beberapa hari ke depan. Badai ini diperkirakan akan menghasilkan angin kencang dengan kecepatan mencapai 35 knot atau 65 kilometer per jam. Di lokasi yang terkena dampak badai, kemungkinan besar akan terjadi hujan lebat, gelombang tinggi, dan banjir bandang. Kami menyarankan agar Anda mengambil langkah-langkah untuk mempersiapkan diri dan mengikuti informasi terbaru dari pihak berwenang. Terima kasih.



In [24]:
from IPython.display import Audio

txt = fair_mms_tts.preprocess_text(txt, text_mapper, hps, lang=lang_code)
stn_tst = text_mapper.get_text(txt, hps)

with torch.no_grad():
    x_tst = stn_tst.unsqueeze(0).to(device)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).to(device)
    hyp = net_g.infer(
        x_tst, x_tst_lengths, noise_scale=.667,
        noise_scale_w=0.8, length_scale=1.0
    )[0][0,0].cpu().float().numpy()

print(f"Generated audio")
Audio(hyp, rate=hps.data.sampling_rate)

ind
text after filtering OOV: berikut adalah laporan cuaca terbaru saat ini ada badai tropis terbentuk di samudra pasifik badai ini diperkirakan akan bergerak ke arah barat laut dan kemungkinan besar akan mempengaruhi indonesia dalam beberapa hari ke depan badai ini diperkirakan akan menghasilkan angin kencang dengan kecepatan mencapai 5 knot atau 65 kilometer per jam di lokasi yang terkena dampak badai kemungkinan besar akan terjadi hujan lebat gelombang tinggi dan banjir bandang kami menyarankan agar anda mengambil langkah-langkah untuk mempersiapkan diri dan mengikuti informasi terbaru dari pihak berwenang terima kasih
Generated audio


In [35]:
commons

<module 'commons' from '/root/vits/commons.py'>

In [17]:
!git -C /content/hurricane-tts/ pull

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), 298 bytes | 298.00 KiB/s, done.
From https://github.com/hammad93/hurricane-tts
   988393b..719f43d  main       -> origin/main
Updating 988393b..719f43d
Fast-forward
 fair_mms_tts.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [30]:
%load_ext autoreload
%autoreload 2